Pada latihan kali ini kita akan melakukan klasifikasi teks multikelas menggunakan lstm.

Pada latihan ini kita akan menggunakan dataset yang berisi sinopsis dari beberapa film Indonesia dan genrenya. Tujuan kita adalah menentukan genre sebuah film berdasarkan sinopsisnya. Dataset dapat Anda unduh pada tautan berikut https://www.kaggle.com/datasets/antoniuscs/imdb-synopsis-indonesian-movies.

Pada cell pertama impor library pandas dan ubah dataset menjadi dataframe. Kemudian buang kolom 'judul_film' karena kita hanya akan menggunakan sinopsis sebagai atribut untuk dilatih pada model.

In [ ]:
import pandas as pd

df = pd.read_csv('imdb_indonesian_movies_2.csv')

df.info()

df = df.drop(columns=['judul_film'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   judul_film          1005 non-null   object
 1   ringkasan_sinopsis  1005 non-null   object
 2   genre               1005 non-null   object
dtypes: object(3)
memory usage: 23.7+ KB


In [ ]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding. Jalankan kode di bawah untuk melakukan one-hot-encoding dan membuat dataframe baru.

In [ ]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis = 1)
df_baru = df_baru.drop(columns=['genre'])
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [ ]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

Lalu, bagi data untuk training dan data untuk testing.

In [ ]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

Kemudian kita ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer. Setelah tokenisasi selesai, kita perlu membuat mengonversi setiap sampel menjadi sequence.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 5000, oov_token = 'x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sequence_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sequence_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sequence_latih)
padded_test = pad_sequences(sequence_test)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. Jangan lupa panggil fungsi compile dan tentukan optimizer serta loss function yang akan dipakai oleh model.

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
           tf.keras.layers.Embedding(input_dim = 5000, output_dim = 16),
           tf.keras.layers.LSTM(64),
           tf.keras.layers.Dense(128, activation = 'relu'),
           tf.keras.layers.Dense(64, activation = 'relu'),
           tf.keras.layers.Dense(5, activation = 'softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Terakhir kita dapat mulai melatih model kita dengan memanggil fungsi fit().

In [ ]:
history = model.fit(padded_latih, 
                    label_latih,
                    epochs = 30,
                    validation_data = (padded_test, label_test),
                    verbose = 2)

Epoch 1/30
26/26 - 29s - loss: 1.6104 - accuracy: 0.1828 - val_loss: 1.6107 - val_accuracy: 0.1692 - 29s/epoch - 1s/step
Epoch 2/30
26/26 - 25s - loss: 1.6068 - accuracy: 0.2177 - val_loss: 1.6090 - val_accuracy: 0.2139 - 25s/epoch - 952ms/step
Epoch 3/30
26/26 - 27s - loss: 1.5773 - accuracy: 0.3346 - val_loss: 1.5787 - val_accuracy: 0.2687 - 27s/epoch - 1s/step
Epoch 4/30
26/26 - 26s - loss: 1.3947 - accuracy: 0.4328 - val_loss: 1.6332 - val_accuracy: 0.2338 - 26s/epoch - 1s/step
Epoch 5/30
26/26 - 25s - loss: 1.1147 - accuracy: 0.5721 - val_loss: 1.8262 - val_accuracy: 0.2637 - 25s/epoch - 959ms/step
Epoch 6/30
26/26 - 25s - loss: 0.6795 - accuracy: 0.7500 - val_loss: 2.2975 - val_accuracy: 0.2886 - 25s/epoch - 944ms/step
Epoch 7/30
26/26 - 24s - loss: 0.3909 - accuracy: 0.8545 - val_loss: 2.4698 - val_accuracy: 0.2736 - 24s/epoch - 936ms/step
Epoch 8/30
26/26 - 24s - loss: 0.2336 - accuracy: 0.9328 - val_loss: 3.2503 - val_accuracy: 0.2935 - 24s/epoch - 939ms/step
Epoch 9/30
26/26 

Akurasi dari model kita menunjukkan terjadinya overfitting karena akurasi pada data testing sangat besar, sedangkan akurasi pada data validasi jauh lebih kecil. Hal ini masih sangat luar biasa karena kita hanya memiliki 1000 buah sampel data!

Selamat! Anda telah paham bagaimana mengembangkan model ML untuk salah satu kasus NLP yaitu klasifikasi teks. Ada telah paham bagaimana memproses teks, serta menggunakan Embedding dan LSTM layer pada arsitektur modul Anda. 

Setelah submodul ini Anda harus menyelesaikan submission pertama Anda tentang NLP agar bisa mengakses materi-materi selanjutnya. Semangat!

In [2]:
text = """This is a sample sentence, showing off the stop words filtration."""
splt = text.split(' ')

splt

['This',
 'is',
 'a',
 'sample',
 'sentence,',
 'showing',
 'off',
 'the',
 'stop',
 'words',
 'filtration.']